## RAG Pipelines - Data Ingestion to Vector DB Pipeline

In [3]:
import os
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [4]:
### Read all the pdf's inside the directory
def process_all_pdfs(base_directory):
    """Process all PDF Files and attach subject metadata based on folder name"""
    all_documents=[]
    base_dir=Path(base_directory)
    # Find all PDF Files recursively
    pdf_files= list(base_dir.glob("**/*.pdf"))

    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents=loader.load()

            subject= pdf_file.parent.name

            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
                doc.metadata['subject']= subject

            all_documents.extend(documents) 
            print(f"  Loaded {len(documents)} pages")

        except Exception as e:
            print(f" Error:{e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")
           


Found 6 PDF files to process

Processing: 3110004 (1).pdf
  Loaded 2 pages

Processing: 3110004.pdf
  Loaded 1 pages

Processing: 3110005 (1).pdf
  Loaded 2 pages

Processing: 3110005.pdf
  Loaded 2 pages

Processing: 3110006 (1).pdf
  Loaded 2 pages

Processing: 3110006.pdf
  Loaded 2 pages

Total documents loaded: 11


In [7]:
all_pdf_documents

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2022-03-15T03:09:34-07:00', 'title': 'Ahmedabad Center', 'author': 'systems', 'moddate': '2022-03-15T03:09:34-07:00', 'source': '..\\data\\BCE\\3110004 (1).pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': '3110004 (1).pdf', 'file_type': 'pdf', 'subject': 'BCE'}, page_content='1 \nSeat No.: ________ Enrolment No.___________ \n \nGUJARAT TECHNOLOGICAL UNIVERSITY  \n                          BE - SEMESTER–1/2 EXAMINATION – WINTER 2021 \nSubject Code:3110004                                                            Date:25/03/2022   \nSubject Name:Basic Civil Engineering   \nTime:10:30 AM TO 01:00 PM                                           Total Marks:70   \nInstructions:  \n1. Attempt all questions.   \n2. Make suitable assumptions wherever necessary.   \n3. Figures to the right indicate full marks.  \n4. Simple and non-programmable scientific calculators are 

In [5]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """S plit documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["n\n","\n"," ",""]
    )
    split_docs=text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")

    return split_docs    

In [6]:
chunks=split_documents(all_pdf_documents)

Split 11 documents into 27 chunks

Example chunk:
Content: 1 
Seat No.: ________ Enrolment No.___________ 
 
GUJARAT TECHNOLOGICAL UNIVERSITY  
                          BE - SEMESTER–1/2 EXAMINATION – WINTER 2021 
Subject Code:3110004                        ...
Metadata: {'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2022-03-15T03:09:34-07:00', 'title': 'Ahmedabad Center', 'author': 'systems', 'moddate': '2022-03-15T03:09:34-07:00', 'source': '..\\data\\BCE\\3110004 (1).pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': '3110004 (1).pdf', 'file_type': 'pdf', 'subject': 'BCE'}


## Embedding and VectorStoreDB

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict,Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str ="all-MiniLM-L6-v2"):
        """
        Initialize the embedding Manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name=model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}") 
            self.model = SentenceTransformer(self.model_name)
            print(f"Model oaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embedding for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """ 
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embedding for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
## Initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager     
 

Loading embedding model: all-MiniLM-L6-v2


c:\Users\Dhruv\RAG Chatbot\venv\Lib\site-packages\transformers\utils\generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Model oaded successfully. Embedding dimension: 384


## VectorStore

In [9]:
class VectorStore:
    """Manages documnet embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str ="../data/vector_store"):
        """
        Initislize the vector store
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name=collection_name
        self.persist_directory=persist_directory
        self.client=None
        self.collection=None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection=self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise    

    def add_documents(self,documents, embeddings):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """

        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vector store...")

        # Prepare data for ChromaDb    
        ids= []
        metadatas= []
        documents_text= []
        embeddings_list= []

        for i, (doc,embedding) in enumerate(zip(documents, embeddings)):
            #Generate unique ID
            doc_id=f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            #Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index']=i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            #Document content
            documents_text.append(doc.page_content)

            #Embedding
            embeddings_list.append(embedding.tolist())

        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )    
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise 

vectorstore=VectorStore()
vectorstore





Vector store initialized. Collection: pdf_documents
Existing documents in collection: 89


In [17]:
chunks

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2022-03-15T03:09:34-07:00', 'title': 'Ahmedabad Center', 'author': 'systems', 'moddate': '2022-03-15T03:09:34-07:00', 'source': '..\\data\\BCE\\3110004 (1).pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': '3110004 (1).pdf', 'file_type': 'pdf', 'subject': 'BCE'}, page_content='1 \nSeat No.: ________ Enrolment No.___________ \n \nGUJARAT TECHNOLOGICAL UNIVERSITY  \n                          BE - SEMESTER–1/2 EXAMINATION – WINTER 2021 \nSubject Code:3110004                                                            Date:25/03/2022   \nSubject Name:Basic Civil Engineering   \nTime:10:30 AM TO 01:00 PM                                           Total Marks:70   \nInstructions:  \n1. Attempt all questions.   \n2. Make suitable assumptions wherever necessary.   \n3. Figures to the right indicate full marks.  \n4. Simple and non-programmable scientific calculators are 

In [10]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

#Store in the vector database
vectorstore.add_documents(chunks,embeddings)


Generating embedding for 27 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:05<00:00,  5.13s/it]

Generated embeddings with shape: (27, 384)
Adding 27 documents to vector store...
Successfully added 27 documents to vector store
Total documents in collection: 116


## Retriever Pipeline From VectorStore

In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector Store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """

        self.vector_store= vector_store
        self.embedding_manager= embedding_manager

    def retrieve(self, query: str,subject: str,top_k: int=5, score_threshold: float = 0.2):
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score thershold
        
        Returns:
            Lost of dictionaries containing retrieved documents and metadata
        """

        print(f"Retrieving documents for query: '{query}")
        print(f"top_k: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k,
                where={'subject':subject}
            )  

            # Process results
            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document , metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance):
                    similarity_score = 1/(1+distance)

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata':metadata,
                            'similarity_score':similarity_score,
                            'distance': distance,
                            'rank': i+1
                        })

                print(f"Retrieval {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)        


In [12]:
rag_retriever

## Integrating VectorDB Context pipeline with LLM Output

In [25]:
## Generate the answer using GROQ_LLM
PROMPT_TEMPLATE = """
You are an experienced university question-paper setter.
USER QUERY:
{user_query}
TASK:
Using ONLY the provided Previous Year Question (PYQ) context,
respond based on the USER REQUEST.

GENERAL RULES:
- Do NOT summarize
- Do NOT explain concepts
- ONLY generate QUESTIONS
- Do NOT add answers
- Use clear numbering

MODE 1: FULL QUESTION PAPER
- If the user asks to generate a question paper or predicted paper,
  generate a COMPLETE, READABLE, and WELL-FORMATTED question paper.
- Maintain an exam-like format.
- This is a PREDICTED paper based on PYQs.

EXAM FORMAT (ONLY FOR MODE 1):
--------------------------------
SECTION A - Very Short Answer (2 marks × 10)
SECTION B - Short Answer (5 marks × 6)
SECTION C - Long Answer (10 marks × 4)
--------------------------------

MODE 2: TOPIC-ONLY QUESTIONS
- If the user asks for questions related to a PARTICULAR TOPIC,
  return ONLY the questions from the PYQ context that are related
  to or contain that topic.
- Do NOT format as a question paper.
- Do NOT include sections, marks, or headings.
- Return a simple numbered list of questions.

INSTRUCTIONS:
1. Use ONLY the given PYQ context.
2. Do NOT invent questions outside the context.
3. Assume suitable data if required.

CONTEXT (PYQs):
{context}

NOW RESPOND ACCORDING TO THE USER REQUEST.
"""

In [26]:
### Simple RAG Pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environments)
groq_api_key="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"

llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.3,max_tokens=1200)

## Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, subject,llm,top_k=3):
    ## retriever the context
    
    results=retriever.retrieve(query,subject=subject,top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    prompt = PROMPT_TEMPLATE.format(
        user_query=query,
        context=context
    )
    
        
    
    
    response=llm.invoke(prompt)
    return response.content

In [29]:
answer=rag_simple("Give me probable question paper",rag_retriever,"BEE",llm)

Retrieving documents for query: 'Give me probable question paper
top_k: 3, Score threshold: 0.2
Generating embedding for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 42.79it/s]

Generated embeddings with shape: (1, 384)
Retrieval 3 documents (after filtering)


In [30]:
print(answer)

USER QUERY:
Give me a probable question paper.

MODE 1: FULL QUESTION PAPER

**GUJARAT TECHNOLOGICAL UNIVERSITY**
**BE - SEMESTER–1/2 EXAMINATION – WINTER 2021**
**Subject Code:3110005**
**Subject Name:Basic Electrical Engineering**
**Date:29/03/2022**
**Time:10:30 AM TO 01:00 PM**
**Total Marks:70**

**SECTION A - Very Short Answer (2 marks × 10)**

1. Define Power Factor.
2. Mention the advantages of MCB over Fuse.
3. Explain the principle of operation of an Alternator.
4. Describe the construction of a DC machine.
5. Write the applications of Auto Transformer.
6. Mention the basic guidelines (Safety Rules) regarding safe handling of electricity.
7. Explain the construction of single phase Transformer.
8. Label the parts of a cable.
9. Mention the disadvantages of Low Power Factor.
10. Write the short technical notes on Battery Maintenance.

**SECTION B - Short Answer (5 marks × 6)**

1. Calculate current, resistance and energy consumed by an oven rated 230V, 1KW when used for 20 hou